# 7. 실전타임: 한국시장에서 버핏류 주식을 찾아보자

[7강의](https://www.youtube.com/watch?v=klnhcMJJS-s) 를 파이썬으로 구현해보겠습니다.

## 설치

Pandas 가 있어야 합니다. 설치가 안되어있으시면 다음 셀을 실행시키세요.

In [1]:
import sys
!{sys.executable} -m pip install pandas xlrd openpyxl

## 다운로드

[네이버 카페 퀀트킹(QuantKing)](https://cafe.naver.com/quantking)에서 최신 엑셀 파일을 다운받으세요.

저는 **QuantKing(2019.01.25).zip** 파일을 받았습니다. 받으면 확장자 **xlsb** 파일이 보입니다. 엑셀로 여셔서 **QuantKing.xlsx** 파일로 다시 저장해주세요.

## 파일 로딩

파일을 열어서 조사해 보겠습니다. Pandas의 read_excel() 이란 함수를 이용하면 열 수 있네요.

In [2]:
import pandas as pd

In [3]:
filepath = 'QuantKing.xlsx'
df = pd.read_excel(filepath, sheet_name='퀀트데이타', skiprows=2).drop('Unnamed: 0', axis=1)
df.head()

,코드 번호,회사명,업종 (대),업종 (소),코스피코스닥,주가 (원),시가총액 (억),상장주식수 (만주),자사주 (만주),자사주 비중 (%),...,18년 2Q.2,18년 3Q.2,18년 4Q(E).2,19년 1Q(E).2,18년4Q 매출액,18년4Q 영업이익,18년4Q 순이익,18년4Q 매출액.1,18년4Q 영업이익.1,18년4Q 순이익.1
0,A005930,삼성전자,반도체 관련장비 및 부품,종합 반도체,코스피,44750,2671478,596978.0,0,0.0,...,109815,129674,100441.00,79168.0,590000.00,108000.00,NaN,631283.0,133354.0,100441.0
1,A000660,SK하이닉스,반도체 관련장비 및 부품,종합 반도체,코스피,74600,543090,72800.0,4400,6.0,...,43299,46936,33962.90,16797.0,99380.81,44300.75,33962.90,101549.0,50660.0,40819.0
2,A005380,현대차,자동차 및 관련부품,완성차,코스피,128500,274564,21367.0,939,4.4,...,7006,2692,-1297.74,7829.0,256695.34,5011.27,-1297.74,261108.0,7611.0,6727.0
3,A068270,셀트리온,제약 및 바이오,바이오,코스피,208500,266772,12795.0,85,0.7,...,840,562,544.00,521.0,NaN,NaN,NaN,2512.0,792.0,544.0
4,A207940,삼성바이오로직스,제약 및 바이오,바이오,코스피,403000,266645,6617.0,0,0.0,...,-96,-301,4362.00,28.0,NaN,NaN,NaN,1393.0,149.0,4362.0


강의에서 사용하는 컬럼만 잘라서 보관하도록 하겠습니다.

In [4]:
df= df[['회사명',
        '시가총액\n(억)', '발표\nPBR', '과거\nPER', '과거\nPSR', '시가\n배당률\n(%)', # 저평가
        '과거\nROE\n(%)', '14년->17년\n3년간 YOY', '단순\n부채비율\n(%)', '배당\n성향\n(%)', # 우량주
        '주가\n변동성', # 변동성
       ]].set_index('회사명')
df.head()

,시가총액 (억),발표 PBR,과거 PER,과거 PSR,시가 배당률 (%),과거 ROE (%),14년->17년 3년간 YOY,단순 부채비율 (%),배당 성향 (%),주가 변동성
회사명,,,,,,,,,,
삼성전자,2671478,1.14,5.62,1.07,1.9,20.3,16.2,39,12.3,0.01720
SK하이닉스,543090,1.16,3.53,1.37,1.3,35.7,75.8,32,6.4,0.02344
현대차,274564,0.40,10.28,0.29,3.1,2.2,8.0,140,20.3,0.01940
셀트리온,266772,10.53,78.90,26.31,0.0,13.3,101.5,31,0.6,0.03486
삼성바이오로직스,266645,7.13,-249.20,50.91,0.0,-2.9,340.8,95,0.0,0.03688


## 저평가 순위

저평가 순위를 매겨보겠습니다.

In [5]:
df = df[df['발표\nPBR'] != 0]
df = df[df['과거\nPER'] != 0]
df = df[df['과거\nPSR'] != 0]

df['1/PBR'] = 1 / df['발표\nPBR']
df['1/PER'] = 1 / df['과거\nPER']
df['1/PSR'] = 1 / df['과거\nPSR']

df['순위 배당수익'] = df['시가\n배당률\n(%)'].rank()
df['순위 PBR'] = df['1/PBR'].rank()
df['순위 PER'] = df['1/PER'].rank()
df['순위 PSR'] = df['1/PSR'].rank()
df['통합 저평가 지표'] = df[['순위 배당수익', '순위 PBR', '순위 PER', '순위 PSR']].mean(axis=1)
df.head()

,시가총액 (억),발표 PBR,과거 PER,과거 PSR,시가 배당률 (%),과거 ROE (%),14년->17년 3년간 YOY,단순 부채비율 (%),배당 성향 (%),주가 변동성,1/PBR,1/PER,1/PSR,순위 배당수익,순위 PBR,순위 PER,순위 PSR,통합 저평가 지표
회사명,,,,,,,,,,,,,,,,,,
삼성전자,2671478,1.14,5.62,1.07,1.9,20.3,16.2,39,12.3,0.01720,0.877193,0.177936,0.934579,1568.5,1010.5,1865.0,920.0,1341.000
SK하이닉스,543090,1.16,3.53,1.37,1.3,35.7,75.8,32,6.4,0.02344,0.862069,0.283286,0.729927,1364.5,990.5,1974.0,754.0,1270.750
현대차,274564,0.40,10.28,0.29,3.1,2.2,8.0,140,20.3,0.01940,2.500000,0.097276,3.448276,1816.5,1891.5,1600.0,1712.0,1755.000
셀트리온,266772,10.53,78.90,26.31,0.0,13.3,101.5,31,0.6,0.03486,0.094967,0.012674,0.038008,461.0,33.0,826.5,40.0,340.125
삼성바이오로직스,266645,7.13,-249.20,50.91,0.0,-2.9,340.8,95,0.0,0.03688,0.140252,-0.004013,0.019643,461.0,55.0,670.0,24.0,302.500


## 우량주 순위

우량주 순위를 매겨보겠습니다.

In [6]:
df['ROE 순위'] = df['과거\nROE\n(%)'].rank()
df['영업이익 성장 순위'] = df['14년->17년\n3년간 YOY'].rank()
df['부채비율 순위'] = df['단순\n부채비율\n(%)'].rank()
df['배당성향 순위'] = df['배당\n성향\n(%)'].rank()
df['통합 우량주 지표'] = df[['ROE 순위', '영업이익 성장 순위', '부채비율 순위', '배당성향 순위']].mean(axis=1)
df.head()

,시가총액 (억),발표 PBR,과거 PER,과거 PSR,시가 배당률 (%),과거 ROE (%),14년->17년 3년간 YOY,단순 부채비율 (%),배당 성향 (%),주가 변동성,...,순위 배당수익,순위 PBR,순위 PER,순위 PSR,통합 저평가 지표,ROE 순위,영업이익 성장 순위,부채비율 순위,배당성향 순위,통합 우량주 지표
회사명,,,,,,,,,,,,,,,,,,,,,
삼성전자,2671478,1.14,5.62,1.07,1.9,20.3,16.2,39,12.3,0.01720,...,1568.5,1010.5,1865.0,920.0,1341.000,1908.5,1096.0,619.5,1271.5,1223.875
SK하이닉스,543090,1.16,3.53,1.37,1.3,35.7,75.8,32,6.4,0.02344,...,1364.5,990.5,1974.0,754.0,1270.750,1989.0,1679.5,510.0,1091.0,1317.375
현대차,274564,0.40,10.28,0.29,3.1,2.2,8.0,140,20.3,0.01940,...,1816.5,1891.5,1600.0,1712.0,1755.000,873.0,915.0,1518.0,1480.5,1196.625
셀트리온,266772,10.53,78.90,26.31,0.0,13.3,101.5,31,0.6,0.03486,...,461.0,33.0,826.5,40.0,340.125,1732.0,1778.0,493.5,1027.0,1257.625
삼성바이오로직스,266645,7.13,-249.20,50.91,0.0,-2.9,340.8,95,0.0,0.03688,...,461.0,55.0,670.0,24.0,302.500,559.0,1961.0,1246.5,563.5,1082.500


## 변동성 순위

변동성 순위를 매겨보겠습니다.

In [7]:
df['변동성 순위'] = df['주가\n변동성'].rank()
df.head()

,시가총액 (억),발표 PBR,과거 PER,과거 PSR,시가 배당률 (%),과거 ROE (%),14년->17년 3년간 YOY,단순 부채비율 (%),배당 성향 (%),주가 변동성,...,순위 PBR,순위 PER,순위 PSR,통합 저평가 지표,ROE 순위,영업이익 성장 순위,부채비율 순위,배당성향 순위,통합 우량주 지표,변동성 순위
회사명,,,,,,,,,,,,,,,,,,,,,
삼성전자,2671478,1.14,5.62,1.07,1.9,20.3,16.2,39,12.3,0.01720,...,1010.5,1865.0,920.0,1341.000,1908.5,1096.0,619.5,1271.5,1223.875,210.5
SK하이닉스,543090,1.16,3.53,1.37,1.3,35.7,75.8,32,6.4,0.02344,...,990.5,1974.0,754.0,1270.750,1989.0,1679.5,510.0,1091.0,1317.375,563.0
현대차,274564,0.40,10.28,0.29,3.1,2.2,8.0,140,20.3,0.01940,...,1891.5,1600.0,1712.0,1755.000,873.0,915.0,1518.0,1480.5,1196.625,310.5
셀트리온,266772,10.53,78.90,26.31,0.0,13.3,101.5,31,0.6,0.03486,...,33.0,826.5,40.0,340.125,1732.0,1778.0,493.5,1027.0,1257.625,1351.0
삼성바이오로직스,266645,7.13,-249.20,50.91,0.0,-2.9,340.8,95,0.0,0.03688,...,55.0,670.0,24.0,302.500,559.0,1961.0,1246.5,563.5,1082.500,1451.5


## 통합 순위

통합 순위를 매겨보겠습니다. 단순히 (저평가 지표 * 0.4 + 우량주지표 * 0.4 + 변동성 지표 * 0.2) 로 계산하시고 정렬하시면 됩니다.

In [8]:
df['통합순위'] = df['통합 저평가 지표'] * 0.4 + df['통합 우량주 지표'] * 0.4 + df['변동성 순위'] * 0.2
df.sort_values(by='통합순위').head(n=10)

,시가총액 (억),발표 PBR,과거 PER,과거 PSR,시가 배당률 (%),과거 ROE (%),14년->17년 3년간 YOY,단순 부채비율 (%),배당 성향 (%),주가 변동성,...,순위 PER,순위 PSR,통합 저평가 지표,ROE 순위,영업이익 성장 순위,부채비율 순위,배당성향 순위,통합 우량주 지표,변동성 순위,통합순위
회사명,,,,,,,,,,,,,,,,,,,,,
에이앤티앤,730,9.48,-17.80,8.11,0.0,-53.2,-9.0,19,0.0,0.00000,...,384.5,114.5,249.000,76.0,469.0,295.0,563.5,350.875,3.5,240.65
마제스타,672,1.93,-11.39,10.03,0.0,-16.9,-82.4,22,0.0,0.00000,...,306.0,85.0,355.250,266.5,11.0,344.5,563.5,296.375,3.5,261.35
싸이토젠,594,17.47,-31.26,84.86,0.0,-55.9,-50.0,0,0.0,0.02214,...,483.0,13.0,241.750,72.0,79.5,7.5,563.5,180.625,470.0,262.95
UCI,484,4.48,-16.13,2.47,0.0,-27.8,-21.5,75,0.0,0.00000,...,370.0,422.5,349.375,183.0,281.5,1064.5,563.5,523.125,3.5,349.70
케이에스피,518,8.63,-6.64,3.05,0.0,-130.0,-51.8,662,0.0,0.00000,...,205.0,323.5,257.875,17.0,74.0,1951.0,563.5,651.375,3.5,364.40
코디,426,2.18,-2.82,3.70,0.0,-77.4,-77.1,243,0.0,0.01231,...,81.0,258.0,322.500,41.0,20.0,1818.5,563.5,610.750,56.0,384.50
파루,853,1.80,-3.07,2.63,0.0,-58.5,-50.1,29,0.0,0.02507,...,84.0,382.5,387.125,68.0,77.5,457.0,563.5,291.500,694.0,410.25
휴맥스홀딩스,430,0.27,-1.39,8.27,0.0,-19.6,-75.4,17,0.0,0.01997,...,32.0,112.0,649.625,241.0,22.0,258.0,563.5,271.125,344.0,437.10
올릭스,3844,6.96,-53.39,3844.00,0.0,-13.0,-66.7,4,0.0,0.03231,...,551.0,3.0,268.750,312.5,33.0,27.0,563.5,234.000,1196.0,440.30
